In [1]:
import sys  
sys.path.insert(0, '/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/')
import numpy as np
path_dir = r"/Users/../Volumes/Affiliations/"

In [2]:
import sys  
import pickle
sys.path.insert(0, '/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/')
import numpy as np
import src.utils
from src.utils import import_dta, import_tracks_dta, successful_creators_followers,\
gen_active_relations, get_fan_interactions_per_week, calculate_avg_monthly_valence,\
gen_active_relations_prob, get_fan_interactions_per_week_prob, stripplot_prob,\
reaction_probability, follower_list, filter_quantile, sample_creators_music,\
gen_outbound_creators
import datetime
from datetime import datetime
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import mstats
pd.plotting.register_matplotlib_converters()
import os

import src.probability_utils
from src.probability_utils import volume_analysis, valence_analysis
import math
import matplotlib.ticker as plticker
from scipy.interpolate import make_interp_spline, BSpline
import seaborn as sns
from numpy import random
from tabulate import tabulate
from IPython.display import display
from tqdm import tqdm

In [3]:
def process_date(date):
    year = date[0:4]
    week = date[6:]
    day = "1"
    date = "{}-{}-1".format(year, week)
    dt = datetime.strptime(date, "%Y-%W-%w")
    return dt

In [4]:
path_dir = r"/Users/../Volumes/Raw/"
music_before = 14
low_success = 0.5
high_success = 0.9
rank_percentiles = False

In [5]:
#we will skip comments for now

#affiliations :follows
#favoritings :likes

tracks = import_tracks_dta(path_dir, "12sample_tracks.dta");
follows_sent = import_dta(path_dir, "12sample_affiliations_sent.dta");
comments_sent = import_dta(path_dir, "12sample_comments_made.dta");
shares_sent = import_dta(path_dir, "12sample_reposts_made.dta");
likes_sent = import_dta(path_dir, "12sample_favoritings_made.dta");
messages_sent = import_dta(path_dir, "12sample_messages_sent.dta");

user_info = import_dta(path_dir, "12sample_user_infos.dta");
user_info.columns = ['user_id', 'type', 'entered_platform']
user_info_2 = import_dta(path_dir, "12sample_1st_deg_user_infos.dta");
user_info_2.columns = ['user_id', 'type', 'entered_platform']
user_info = pd.concat([user_info, user_info_2]);
user_info.drop_duplicates(subset='user_id', keep='first', inplace=True, ignore_index=False);


########
follows_received = import_dta(path_dir, "12sample_affiliations_received.dta");
comments_received = import_dta(path_dir, "12sample_comments_received.dta");
shares_received = import_dta(path_dir, "12sample_reposts_received.dta");
likes_received = import_dta(path_dir, "12sample_favoritings_received.dta");
messages_received = import_dta(path_dir, "12sample_messages_received.dta");

########
follows = import_dta(path_dir, "12sample_affiliations.dta");
comments = import_dta(path_dir, "12sample_comments.dta");
shares = import_dta(path_dir, "12sample_reposts.dta");
likes = import_dta(path_dir, "12sample_favoritings.dta");
messages = import_dta(path_dir, "12sample_messages.dta");

tracks_first_month = sample_creators_music(tracks, music_before)
mask = tracks_first_month.track_available == 1
tracks_first_month = tracks_first_month[mask]

mask = follows_received.contact_id.isin(tracks_first_month.user_id)
follows_received_30 = follows_received[mask]
followers_per_creator_60 = successful_creators_followers(follows_received_30, 60, tracks_first_month)
followers_per_creator_60_static = successful_creators_followers(follows_received_30, 60, tracks_first_month, always_same_batches = True, 
                                                        perc1 = low_success, perc2 = high_success)

%%%%%%%%%% 12sample_tracks.dta %%%%%%%%%%
(56262, 7)
%%%%%%%%%% 12sample_affiliations_sent.dta %%%%%%%%%%
(800913, 3)
%%%%%%%%%% 12sample_comments_made.dta %%%%%%%%%%
(29258, 4)
%%%%%%%%%% 12sample_reposts_made.dta %%%%%%%%%%
(179329, 4)
%%%%%%%%%% 12sample_favoritings_made.dta %%%%%%%%%%
(527701, 4)
%%%%%%%%%% 12sample_messages_sent.dta %%%%%%%%%%
(11091, 3)
%%%%%%%%%% 12sample_user_infos.dta %%%%%%%%%%
(35000, 3)
%%%%%%%%%% 12sample_1st_deg_user_infos.dta %%%%%%%%%%
(670746, 3)
%%%%%%%%%% 12sample_affiliations_received.dta %%%%%%%%%%
(432503, 3)
%%%%%%%%%% 12sample_comments_received.dta %%%%%%%%%%
(21386, 4)
%%%%%%%%%% 12sample_reposts_received.dta %%%%%%%%%%
(83013, 4)
%%%%%%%%%% 12sample_favoritings_received.dta %%%%%%%%%%
(286903, 4)
%%%%%%%%%% 12sample_messages_received.dta %%%%%%%%%%
(17364, 3)
%%%%%%%%%% 12sample_affiliations.dta %%%%%%%%%%
(1233416, 3)
%%%%%%%%%% 12sample_comments.dta %%%%%%%%%%
(50644, 4)
%%%%%%%%%% 12sample_reposts.dta %%%%%%%%%%
(262342, 4)
%%%%%%%%%% 12sam

In [6]:
d = {}
for i in range(6):
   d[str(i)] = pd.read_pickle('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/{}.pkl'.format(i)) 
   d[str(i)]['created_at'] =  pd.to_datetime(d[str(i)]['created_at'])#.apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))

In [7]:
data = pd.concat([d['0'], d['1'], d['2'], d['3'], d['4'], d['5']])

In [8]:
#data['created_at'] = pd.to_datetime(data['created_at'])

In [9]:
data.set_index('created_at', inplace = True)

In [10]:
#outdegree = data.groupby('sender_id').agg(following=('contact_id', pd.Series.nunique))
#outdegree_info = user_info_2.merge(outdegree, left_on = 'user_id', right_on = 'sender_id', how= 'left')
#outdegree_info.loc[outdegree_info.following.isnull(), 'following'] = 0 

In [11]:
#outdegree_info['decile'] = pd.qcut(outdegree_info['following'], 10, labels=False)

In [12]:
#outdegree_info.head(10)

In [13]:
#outdegree_info['decile'].value_counts()

In [14]:
actions_sent = gen_outbound_creators(follows_sent, shares_sent, likes_sent, comments_sent, tracks,
                                     messages_sent = messages_sent, filter_creators = True)
mask = actions_sent.user_id.isin(tracks_first_month.user_id.unique())
actions_sent = actions_sent[mask]

In [15]:
#filter only outbound activities before the follow is received
print('Obtaining fanbase...')
follows_received.columns = ['fan_id', 'user_id', 'date_sent']
followers = follower_list(follows_received)
actions_sent = actions_sent.merge(followers, right_on = ['user_id', 'fan_id'],
                                      left_on = ['user_id', 'fan_id'], how = 'left')

Obtaining fanbase...


In [16]:
mask = (actions_sent.date_sent < actions_sent.follower_since) | (actions_sent.follower_since.isnull())
actions_sent_non_followers =  actions_sent[mask]
actions_sent_non_followers['week_yr_date'] = actions_sent_non_followers.week_yr.apply(lambda x: process_date(x))

/var/folders/f2/cgjzt69n5hlgmtsm36p1pztw0000gn/T/ipykernel_6480/3898105865.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actions_sent_non_followers['week_yr_date'] = actions_sent_non_followers.week_yr.apply(lambda x: process_date(x))


# MAB

In [17]:
def create_trials_rewards(df):
    '''
    Creates reward counts, trial counts and keys to be used in the thompson sampling function
    '''
    Sent = df.groupby('outbound_activity', as_index = True).size().to_frame(name='sent')\
    .reset_index()
    Reward = df.groupby('outbound_activity', as_index = True)\
    .agg(b_Reward = ('reward', 'sum')).reset_index()


    keys  = Sent.outbound_activity

    return np.array(Sent.sent), np.array(Reward.b_Reward), keys

In [18]:
def update_1_step(Sent, Reward):
    
    alpha = Reward + 1 
    beta = (Sent-Reward)+1

    mean = alpha/(alpha+beta)
    variance = (alpha*beta)/((alpha+beta+1)*((alpha+beta)**2))

    return np.around(mean,3), np.around(np.sqrt(variance),3)

In [19]:
def jitter(a_series, noise_reduction=1000):
    return (np.random.random(len(a_series))*a_series.std()/noise_reduction)-(a_series.std()/(2*noise_reduction))

In [20]:
def create_deciles(data):
    
    mask = (data['size']>0)
    data.loc[~mask, 'decile'] = 0 
    cutpoints = np.percentile(data[mask]['size'], np.arange(1,100,10)) 
    
    for i in range(len(cutpoints)):
        low = cutpoints[i]
        try:
            high = cutpoints[i+1]
        except:
            high = 100000
        
        mask2 = (data[mask]['size']>=low) & (data[mask]['size'] <high) & (data['size']>0)
        data.loc[mask2,'decile'] = i + 1 
    return data

In [21]:
def real_time(df, outdegree_df, interval, rank_percentiles = rank_percentiles):
    
    d = {}
    
    for user_id in df.user_id.unique():
        
        if rank_percentiles:
        
            experiment = {
             '1':{'Sent':0,'Reward':0}, 
             '2':{'Sent':0,'Reward':0},
             '3':{'Sent':0,'Reward':0},
             '4':{'Sent':0,'Reward':0},
             '5':{'Sent':0,'Reward':0}, 
             '6':{'Sent':0,'Reward':0},
             '7':{'Sent':0,'Reward':0},
             '8':{'Sent':0,'Reward':0},
             '9':{'Sent':0,'Reward':0},
             '10':{'Sent':0,'Reward':0}}
        else:
            
            experiment = {'0':{'Sent':0,'Reward':0},
             '1':{'Sent':0,'Reward':0}, 
             '2':{'Sent':0,'Reward':0},
             '3':{'Sent':0,'Reward':0},
             '4':{'Sent':0,'Reward':0},
             '5':{'Sent':0,'Reward':0}, 
             '6':{'Sent':0,'Reward':0},
             '7':{'Sent':0,'Reward':0},
             '8':{'Sent':0,'Reward':0},
             '9':{'Sent':0,'Reward':0},
             '10':{'Sent':0,'Reward':0}}
            
        if rank_percentiles:
        
            dataset_mean = pd.DataFrame({'trials':range(1,df.loc[df['user_id'] == user_id,:].shape[0]+1),                     
              '1': 0,
              '2': 0,
              '3': 0,
              '4': 0,
              '5': 0,
              '6': 0,
              '7': 0,
              '8': 0,
              '9': 0,
              '10': 0})
        
        else:
             
            dataset_mean = pd.DataFrame({'trials':range(1,df.loc[df['user_id'] == user_id,:].shape[0]+1),                      
              '0': 0,
              '1': 0,
              '2': 0,
              '3': 0,
              '4': 0,
              '5': 0,
              '6': 0,
              '7': 0,
              '8': 0,
              '9': 0,
              '10': 0}) 
            
        if rank_percentiles:    
    
            dataset_std = pd.DataFrame({'trials':range(1,df.loc[df['user_id'] == user_id,:].shape[0]+1),                       
              '1': 0,
              '2': 0,
              '3': 0,
              '4': 0,
              '5': 0,
              '6': 0,
              '7': 0,
              '8': 0,
              '9': 0,
              '10': 0}) 
        
        else:
            dataset_std = pd.DataFrame({'trials':range(1,df.loc[df['user_id'] == user_id,:].shape[0]+1),     
              '0': 0,
              '1': 0,
              '2': 0,
              '3': 0,
              '4': 0,
              '5': 0,
              '6': 0,
              '7': 0,
              '8': 0,
              '9': 0,
              '10': 0}) 
    
        d[user_id] = {'experiment': experiment,'dataset_std': dataset_std, 'dataset_mean': dataset_mean, 'j' : 0}
        

    date_l = min(df.date_sent)
    old_outdegree = outdegree_df.iloc[:
                  outdegree_df.index.values.searchsorted(np.datetime64(date_l))].groupby('sender_id', as_index = False).size()
    old_outdegree = user_info_2.merge(old_outdegree, left_on = 'user_id', right_on = 'sender_id', how= 'left')
    old_outdegree.loc[old_outdegree['size'].isnull(), 'size'] = 0
    old_outdegree = old_outdegree[['user_id', 'size', 'entered_platform']].set_index('user_id')
    mask = old_outdegree['entered_platform'] <= date_l
    if rank_percentiles:
        old_outdegree['decile'] = pd.qcut(old_outdegree.loc[mask,'size'].rank(method='first'), 10, labels=False, duplicates='drop') + 1
    else:
        old_outdegree.loc[mask, 'decile'] = create_deciles(old_outdegree.loc[mask])
    
    old_outdegree['decile'] = old_outdegree[['decile']]  
    
    for i in tqdm(range(df.shape[0])):
        
        user_id = df.iloc[i].user_id
        fan_id = df.iloc[i].fan_id
        j = d[user_id]['j']
        d[user_id]['j'] = j + 1
        
        if i%interval == 0:
            date_h = df.iloc[i,:]['date_sent']
            new_outdegree = outdegree_df.iloc[outdegree_df.index.values.searchsorted(np.datetime64(date_l)):
                  outdegree_df.index.values.searchsorted(np.datetime64(date_h))].groupby('sender_id', as_index = False).size()
        
            old_outdegree['size'] = old_outdegree['size'].add(new_outdegree.set_index('sender_id')['size'], fill_value = 0)
            mask = old_outdegree['entered_platform'] <= date_h
            if rank_percentiles:
                old_outdegree['decile'] = pd.qcut(old_outdegree.loc[mask,'size'].rank(method='first'), 10, labels=False, duplicates='drop') + 1
            else:
                old_outdegree.loc[mask, 'decile'] = create_deciles(old_outdegree.loc[mask])
            old_outdegree['decile'] = old_outdegree[['decile']]  
            date_l = date_h
            
        if i%1000 == 0:
            print(old_outdegree.decile.value_counts(normalize=True))
        
        try:
       
            d[user_id]['experiment'][str(int(old_outdegree.loc[fan_id]['decile']))]['Sent'] += 1
            d[user_id]['experiment'][str(int(old_outdegree.loc[fan_id]['decile']))]['Reward'] += df.iloc[i, -1]
            Sent = np.array([a[1]['Sent'] for a in d[user_id]['experiment'].items()])
            Reward = np.array([a[1]['Reward'] for a in d[user_id]['experiment'].items()])

            mean, std = update_1_step(Sent, Reward)

            d[user_id]['dataset_mean'].loc[j, ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']] = np.transpose(mean)
            d[user_id]['dataset_std'].loc[j, ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']] = np.transpose(std)

            print('.', end='')
        
        except:
            print('x', end ='')
        
    for user_id in df.user_id.unique(): 
        d[user_id]['dataset_mean'] = pd.concat([d[user_id]['dataset_mean'], df.loc[df['user_id'] == user_id].reset_index()['date_sent']], axis = 1)
        
        
    return d

# Real time updating

In [22]:
actions_sent = gen_outbound_creators(follows_sent, shares_sent, likes_sent, comments_sent, tracks,
                                     messages_sent = messages_sent, filter_creators = False)

In [23]:
mask = actions_sent.user_id.isin(tracks_first_month.user_id) 
actions_sent = actions_sent[mask]
actions_sent_filtered = actions_sent.merge(followers_per_creator_60_static, left_on = 'user_id', right_on = 'user_id', how = "inner")

In [24]:
#filter only outbound activities before the follow is received
print('Obtaining fanbase...')
follows_received.columns = ['fan_id', 'user_id', 'date_sent']
followers = follower_list(follows_received)
actions_sent_filtered = actions_sent_filtered.merge(followers, right_on = ['user_id', 'fan_id'],
                                      left_on = ['user_id', 'fan_id'], how = 'left')

mask = (actions_sent_filtered.date_sent < actions_sent_filtered.follower_since) | (actions_sent_filtered.follower_since.isnull())
actions_sent_filtered_non_followers =  actions_sent_filtered[mask]
actions_sent_filtered_non_followers['week_yr_date'] = actions_sent_filtered_non_followers.week_yr.apply(lambda x: process_date(x))

Obtaining fanbase...


/var/folders/f2/cgjzt69n5hlgmtsm36p1pztw0000gn/T/ipykernel_6480/1142436393.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actions_sent_filtered_non_followers['week_yr_date'] = actions_sent_filtered_non_followers.week_yr.apply(lambda x: process_date(x))


In [25]:
mask = actions_sent_filtered_non_followers['fan_id'].isin(data.sender_id.unique())
actions_sent_filtered_non_followers = actions_sent_filtered_non_followers.loc[mask]

In [26]:
import datetime
# Target Creation
delta = datetime.timedelta(days = 1)
mask = (actions_sent_filtered_non_followers['follower_since'] <= (actions_sent_filtered_non_followers['date_sent'] + delta))

actions_sent_filtered_non_followers.loc[mask, 'reward'] = 1
mask = actions_sent_filtered_non_followers['reward'].isnull()
actions_sent_filtered_non_followers.loc[mask, 'reward'] = 0

/var/folders/f2/cgjzt69n5hlgmtsm36p1pztw0000gn/T/ipykernel_6480/1365517025.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actions_sent_filtered_non_followers.loc[mask, 'reward'] = 1


In [27]:
actions_sent_filtered_non_followers.sort_values(by = 'date_sent', inplace = True)


#filter first 2 months
delta = datetime.timedelta(days = 30)

mask = actions_sent_filtered_non_followers.date_sent < (min(actions_sent_filtered_non_followers['date_sent']) + delta)
                                          
actions_sent_filtered_non_followers = actions_sent_filtered_non_followers[mask]    

mask = actions_sent_filtered_non_followers.high_low == 'high'
successful_df = actions_sent_filtered_non_followers[mask]

mask = actions_sent_filtered_non_followers.high_low == 'low'
unsuccessful_df = actions_sent_filtered_non_followers[mask]

/var/folders/f2/cgjzt69n5hlgmtsm36p1pztw0000gn/T/ipykernel_6480/376480209.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actions_sent_filtered_non_followers.sort_values(by = 'date_sent', inplace = True)


In [28]:
dict_successful_df =  real_time(successful_df, data, 10)
with open('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/dict_sucessful_df.pkl', 'wb') as handle:
    pickle.dump(dict_successful_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

/var/folders/f2/cgjzt69n5hlgmtsm36p1pztw0000gn/T/ipykernel_6480/3450947467.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[~mask, 'decile'] = 0
  0%|                                                                                                                                   | 1/16512 [00:01<8:40:47,  1.89s/it]

0.0     0.336367
1.0     0.070809
5.0     0.070723
6.0     0.067603
8.0     0.067549
9.0     0.066851
2.0     0.066728
7.0     0.066555
4.0     0.066122
3.0     0.060966
10.0    0.059727
Name: decile, dtype: float64
..........

  0%|                                                                                                                                  | 11/16512 [00:03<1:22:23,  3.34it/s]

..........

  0%|▏                                                                                                                                 | 21/16512 [00:05<1:05:12,  4.22it/s]

...

..........

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 16501/16512 [51:41<00:02,  5.50it/s]

..........

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16512/16512 [51:42<00:00,  5.32it/s]

..

In [29]:
dict_unsuccessful_df =  real_time(unsuccessful_df, data, 3)
with open('/Users/caiorego/Desktop/BDS/RA/Seeding-Bandits/dict_unsucessful_df.pkl', 'wb') as handle:
    pickle.dump(dict_unsuccessful_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

/var/folders/f2/cgjzt69n5hlgmtsm36p1pztw0000gn/T/ipykernel_6480/3450947467.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[~mask, 'decile'] = 0
  0%|                                                                                                                                    | 1/9831 [00:01<4:47:41,  1.76s/it]

0.0     0.336355
1.0     0.070811
5.0     0.070724
6.0     0.067604
8.0     0.067550
9.0     0.066852
2.0     0.066732
7.0     0.066556
4.0     0.066123
3.0     0.060964
10.0    0.059728
Name: decile, dtype: float64
...

  0%|                                                                                                                                    | 4/9831 [00:03<2:16:56,  1.20it/s]

...

  0%|                                                                                                                                    | 7/9831 [00:05<1:56:58,  1.40it/s]

...

...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 9826/9831 [1:38:42<00:03,  1.40it/s]

...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9831/9831 [1:38:44<00:00,  1.66it/s]

...

In [30]:
len(dict_successful_df.keys())

208

In [31]:
len(dict_unsuccessful_df.keys())

636

#to do: change code so each interaction updates a creator's Mab
1. class mab? Dict?

# Report estimates with standard errors

In [32]:
def div0( a, b, fill=np.inf ):
    """ a / b, divide by 0 -> `fill`
        div0( [-1, 0, 1], 0, fill=np.nan) -> [nan nan nan]
        div0( 1, 0, fill=np.inf ) -> inf
    """
    with np.errstate(divide='ignore', invalid='ignore'):
        c = np.true_divide( a, b )
    if np.isscalar( c ):
        return c if np.isfinite( c ) \
            else fill
    else:
        c[ ~ np.isfinite( c )] = fill
        return c

In [33]:
#add median interactions, mean interactions, std mean interactions, unique users per user group

In [34]:
from collections import Counter
def mean_std_estimates(dic, rank_percentiles = rank_percentiles):

    if rank_percentiles:
        init = 1
        dataset_last_mean = pd.DataFrame(index = range(len(dic)), columns = ['user_id', 'trials', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
        dataset_last_std = pd.DataFrame(index = range(len(dic)), columns = ['user_id', 'trials', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
    else:
        init = 0
        dataset_last_mean = pd.DataFrame(index = range(len(dic)), columns = ['user_id', 'trials', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
        dataset_last_std = pd.DataFrame(index = range(len(dic)), columns = ['user_id', 'trials', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
    for i,key in enumerate(dic):
        if rank_percentiles:
            dataset_last_mean.iloc[i, 1:] = dic[key]['dataset_mean'][['trials', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']].iloc[-1]
        else:
            dataset_last_mean.iloc[i, 1:] = dic[key]['dataset_mean'][['trials', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']].iloc[-1]
        dataset_last_mean.iloc[i, 0] = key
    
        dataset_last_std.iloc[i, 1:] = dic[key]['dataset_std'].iloc[-1]
        dataset_last_std.iloc[i, 0] = key
        
    dicts = [dic[key]['experiment'] for key in dic.keys()]    
    countdic = {}
    for i in dicts[0].keys():
        c = Counter()
        for d in dicts:
            c.update(d[i])
        countdic[i] = c
    
    prob_df = pd.DataFrame(index = range(0,len(dataset_last_mean)-1), columns = ['Decile', 'Trials', 'Rewards'])
    
    for key, value in countdic.items():
        prob_df.iloc[int(key)-init]['Decile'] = key
        prob_df.iloc[int(key)-init]['Trials'] = value['Sent']
        prob_df.iloc[int(key)-init]['Rewards'] = value['Reward']
        
    prob_df.set_index('Decile', inplace = True)
    
    df_trial_info = pd.DataFrame(index = ["{:d}".format(x) for x in range(init,11)] , columns = ['Unique_creators', 'trials'])
    
    for j in range(init,11):
        df_trial_info.iloc[j-init]['trials'] = list()
        df_trial_info.iloc[j-init]['Unique_creators'] = 0
    for i in range(len(dicts)):
        for j in range(init,11):
            if dicts[i][str(j)]['Sent'] != 0:
                df_trial_info.iloc[j-init]['Unique_creators'] = df_trial_info.iloc[j-init]['Unique_creators']+1
            df_trial_info.iloc[j-init]['trials'].extend([dicts[i][str(j)]['Sent']])
    df_trial_info['Median_trials'] = df_trial_info.trials.apply(lambda x: np.median(x))
    df_trial_info['Mean_trials'] = df_trial_info.trials.apply(lambda x: np.mean(x))
    df_trial_info['Std_trials'] = df_trial_info.trials.apply(lambda x: np.std(x))
    
    if rank_percentiles:
        summary = pd.DataFrame(dataset_last_mean[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
        ].mean(), columns = ['Mean Estimate']).merge(
        pd.DataFrame(dataset_last_std[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
        ].mean(), columns = ['Std Estimate']), left_index=True, right_index=True).merge(
    prob_df, left_index=True, right_index=True).merge(df_trial_info[['Unique_creators', 'Median_trials', 'Mean_trials', 'Std_trials']
        ],left_index=True, right_index=True).reindex(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
        
    else:
        summary = pd.DataFrame(dataset_last_mean[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
        ].mean(), columns = ['Mean Estimate']).merge(
        pd.DataFrame(dataset_last_std[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
        ].mean(), columns = ['Std Estimate']), left_index=True, right_index=True).merge(
    prob_df, left_index=True, right_index=True).merge(df_trial_info[['Unique_creators', 'Median_trials', 'Mean_trials', 'Std_trials']
        ],left_index=True, right_index=True).reindex(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])
        
    return dataset_last_mean, dataset_last_std, summary

In [35]:
####################################################################################

In [36]:
####################################################################################

In [37]:
dic = dict_successful_df
sucessful_df_mean, sucessful_df_std, summary_s = mean_std_estimates(dic)

In [38]:
summary_s

,Mean Estimate,Std Estimate,Trials,Rewards,Unique_creators,Median_trials,Mean_trials,Std_trials
0,0.237875,0.151067,2179,94.0,166,4.0,10.475962,17.636811
1,0.288692,0.189308,918,12.0,142,2.0,4.413462,7.397519
2,0.281221,0.186452,932,21.0,151,2.0,4.480769,7.364782
3,0.289365,0.185082,936,34.0,159,2.0,4.500000,7.489095
4,0.296913,0.185726,1109,49.0,149,2.0,5.331731,9.515703
5,0.277082,0.179106,1268,57.0,155,3.0,6.096154,11.713749
6,0.301423,0.184596,1313,81.0,149,2.0,6.312500,13.161476
7,0.313899,0.191019,1385,119.0,143,2.0,6.658654,16.430033
8,0.326755,0.197519,1531,131.0,128,2.0,7.360577,20.425281
9,0.346620,0.203529,1997,247.0,122,1.0,9.600962,30.359851


In [39]:
dic = dict_unsuccessful_df
unsucessful_df_mean, unsucessful_df_std, summary_u = mean_std_estimates(dic)

In [40]:
unsucessful_df_mean.shape

(636, 13)

In [41]:
summary_u

,Mean Estimate,Std Estimate,Trials,Rewards,Unique_creators,Median_trials,Mean_trials,Std_trials
0,0.301236,0.200980,1951,15.0,450,2.0,3.067610,4.313591
1,0.363657,0.232881,995,4.0,345,1.0,1.564465,2.464961
2,0.376162,0.238847,841,7.0,330,1.0,1.322327,2.106042
3,0.372107,0.236981,907,6.0,337,1.0,1.426101,2.396121
4,0.371022,0.234961,1016,8.0,318,0.5,1.597484,2.740609
5,0.387742,0.243613,721,9.0,310,0.0,1.133648,1.737326
6,0.384288,0.242642,757,6.0,313,0.0,1.190252,1.916487
7,0.388631,0.242486,851,7.0,283,0.0,1.338050,2.421185
8,0.394574,0.246657,724,0.0,267,0.0,1.138365,2.093650
9,0.410008,0.252824,588,2.0,232,0.0,0.924528,1.696453


In [42]:
import seaborn as sns

In [43]:
def plot_estimates_dist(dataset, num, sender):
    
    dataset.columns = ['user_id', 'trials', 'arm1', 'arm2', 'arm3', 'arm4', 
                                     'arm5', 'arm6', 'arm7', 'arm8', 'arm9', 'arm10']
    
    to_plot = pd.wide_to_long(dataset, stubnames='arm', i = ['user_id'], j='arm_num')
    to_plot.reset_index(inplace = True)
    to_plot['arm'] = to_plot['arm'].astype(float)

    plt.figure(figsize=(10,7))
    sns.boxplot(x = to_plot.arm_num, y = to_plot.arm).set(
        title="Figure {}:Distribution of Expected rewards per arm. Sender: {}".format(num, sender),
        xlabel="Arm (decile)", ylabel="Expected Reward"
    )
    plt.annotate('''Note: each expected reward estimate is the mean of the beta distribution updated
using arm pulls and observed rewards.'''
             , (0,0), (0, -35), xycoords='axes fraction', textcoords='offset points', va='top', fontsize=14);

In [44]:
plot_estimates_dist(sucessful_df_mean, '17' , 'Successful Creators', 'Creators')

TypeError: plot_estimates_dist() takes 3 positional arguments but 4 were given

In [ ]:
plot_estimates_dist(unsucessful_df_mean, '18' , 'Unsuccessful Creators')

In [ ]:
np.sqrt(variance)

In [ ]:
mask = actions_sent_filtered_non_followers.high_low == 'low'
actions_sent_filtered_non_followers[mask].reward.sum()

# Creator behavior

Weekly share of actions directed to each membership

In [ ]:
def weekly_membership(df, outdegree_df, interval, rank_percentiles = rank_percentiles):

    df_copy = df.copy()
    df_copy['receiver_membership'] = 0
    
    date_l = min(df.date_sent)
    old_outdegree = outdegree_df.iloc[:
                  outdegree_df.index.values.searchsorted(np.datetime64(date_l))].groupby('sender_id', as_index = False).size()
    old_outdegree = user_info_2.merge(old_outdegree, left_on = 'user_id', right_on = 'sender_id', how= 'left')
    old_outdegree.loc[old_outdegree['size'].isnull(), 'size'] = 0 
    old_outdegree = old_outdegree[['user_id', 'size', 'entered_platform']].set_index('user_id')
    
    mask = old_outdegree['entered_platform'] <= date_l
    if rank_percentiles:
        old_outdegree['decile'] = pd.qcut(old_outdegree.loc[mask,'size'].rank(method='first'), 10, labels=False, duplicates='drop') + 1
    else:
        old_outdegree.loc[mask, 'decile'] = create_deciles(old_outdegree.loc[mask])
    old_outdegree['decile'] = old_outdegree[['decile']]  
    
    for i in tqdm(range(df.shape[0])):
        
        fan_id = df.iloc[i].fan_id
        
        if i%interval == 0:
            date_h = df.iloc[i,:]['date_sent']
            new_outdegree = outdegree_df.iloc[outdegree_df.index.values.searchsorted(np.datetime64(date_l)):
                  outdegree_df.index.values.searchsorted(np.datetime64(date_h))].groupby('sender_id', as_index = False).size()
        
            old_outdegree['size'] = old_outdegree['size'].add(new_outdegree.set_index('sender_id')['size'], fill_value = 0)
            mask = old_outdegree['entered_platform'] <= date_h
            if rank_percentiles:
                old_outdegree['decile'] = pd.qcut(old_outdegree.loc[mask,'size'].rank(method='first'), 10, labels=False, duplicates='drop') + 1
            else:
                old_outdegree.loc[mask, 'decile'] = create_deciles(old_outdegree.loc[mask])
            old_outdegree['decile'] = old_outdegree[['decile']]  
            date_l = date_h
        
        try:
            df_copy.iloc[i,-1] = (int(old_outdegree.loc[fan_id]['decile']))
        
            print('.', end='')
        
        except:
            print('x', end ='')
        
        
    return df_copy

In [ ]:
successful_creators_receiver_memb = weekly_membership(successful_df, data, 10)

In [ ]:
unsuccessful_creators_receiver_memb = weekly_membership(unsuccessful_df, data, 5)

In [ ]:
unsuccessful_creators_receiver_memb.date_sent

In [ ]:
def plot_learning(data, number, title, rank_percentiles = rank_percentiles):
    
    data["day_yr_date"] = data.date_sent.dt.normalize()
    
    data_grouped = data.groupby("day_yr_date", as_index = False).size()
    data.columns = ['user_id', 'fan_id', 'date_sent', 'outbound_activity',
       'interaction_week', 'interaction_year', 'week_yr', 'followers',
       'high_low', 'follower_since', 'week_yr_date', 'reward',
       'Receiver Type', 'day_yr_date']
    
    lim_inf = min(data.date_sent)
    lim_sup = max(data.date_sent)
    
    fig, ax = plt.subplots(figsize=(15, 9))
    if rank_percentiles:
        column_list = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
        hue_order_l = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
        
    else:
        column_list = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
        hue_order_l = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

    sns.set(rc={'figure.figsize':(12,8)})
    sns.set_style("white")
    sns.kdeplot(data=data,
            x="day_yr_date",
            hue='Receiver Type',
            hue_order= hue_order_l,
            multiple="fill",
            #weights="Val",
            bw_adjust = 1)\
    .set(xlim=(lim_inf, lim_sup), title='Figure {}: {} - Share of trials per receiver type.'.format(number, title), xlabel='Date',
     ylabel='Share')
    
    ax2 = plt.twinx()
    sns.lineplot(data=data_grouped,
                 x = "day_yr_date", 
                 y = "size", 
                 color="black", 
                 ax=ax2, 
                 legend = 'auto',
                 linewidth = 2)\
    .set(ylabel='Total Promotional Actions', yscale="log")
    plt.annotate('''Note: each color corresponds to a receiver type. The black line is the total number of actions per day.'''
             , (0,0), (0, -35), xycoords='axes fraction', textcoords='offset points', va='top', fontsize=14);
    
    #ax.legend(title='Receiver type', loc='best', labels = ['10', '9', '8', '7', '6', '5', '4', '3', '2', '1', '0']);

In [ ]:
delta = datetime.timedelta(days = 14)
mask = successful_creators_receiver_memb['date_sent'] < (min(successful_creators_receiver_memb['date_sent']) + delta)
plot_learning(successful_creators_receiver_memb[mask], 19, 'Successful Creators')

In [ ]:
delta = datetime.timedelta(days = 14)
mask = unsuccessful_creators_receiver_memb['date_sent'] < (min(unsuccessful_creators_receiver_memb['date_sent']) + delta)
plot_learning(unsuccessful_creators_receiver_memb[mask], 20, 'Unsuccessful Creators')


In [ ]:
actions_sent_non_followers.groupby('week_yr_date', as_index=False).size()['size'].plot()

In [ ]:
outdegree_df = data
date_l = min(successful_df.date_sent)
old_outdegree = outdegree_df.iloc[:
                  outdegree_df.index.values.searchsorted(np.datetime64(date_l))].groupby('sender_id', as_index = False).size()
old_outdegree = user_info_2.merge(old_outdegree, left_on = 'user_id', right_on = 'sender_id', how= 'left')
old_outdegree.loc[old_outdegree['size'].isnull(), 'size'] = 0 
old_outdegree = old_outdegree[['user_id', 'size', 'entered_platform']].set_index('user_id')   


In [ ]:
mask = (old_outdegree['size']>0)
old_outdegree.loc[~mask, 'decile'] = 0 

In [ ]:
old_outdegree

In [ ]:
mask = (old_outdegree['entered_platform'] <= date_l)

In [ ]:
x=create_deciles(old_outdegree[mask])

In [ ]:
x